In [ ]:
# default_exp core

# ihu

> Evolve any unitary using solve_ivp.

In [ ]:
#hide
from nbdev.showdoc import *
from scipy.integrate import solve_ivp
from numpy.random import random_sample
from numpy import array